<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW4/HW4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pycaret
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *

def simplify_ages(df):
    df['Age'] = df['Age'].fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young_Adult', 'Adult', 'Senior']
    categories = pd.cut(df['Age'], bins, labels=group_names)
    df['Age'] = categories
    return df

def simplify_cabins(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0])
    return df

def simplify_fares(df):
    df['Fare'] = df['Fare'].fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df['Fare'], bins, labels=group_names)
    df['Fare'] = categories
    return df

def format_name(df):
    df['Lname'] = df['Name'].apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df['Name'].apply(lambda x: x.split(' ')[1])
    return df

def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = format_name(df)
    df = drop_features(df)
    return df

def rename_features(df):
    df.columns = df.columns.str.replace(r'[^\w]', '_', regex=True)
    return df

# 主程序
if __name__ == "__main__":
    # 載入數據集
    data_train = pd.read_csv('./train.csv')  # 將路徑改為實際數據的位置

    # 轉換特徵
    data_train = transform_features(data_train)
    data_train = rename_features(data_train)

    # 分割數據
    train_data, test_data = train_test_split(data_train.drop(['PassengerId'], axis=1), random_state=100, train_size=0.8)

    # 設定 PyCaret
    clf1 = setup(data=train_data,
                 target='Survived',
                 categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'])

    # 比較模型
    compare_models(fold=5)

,Description,Value
0,Session id,3710
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(712, 10)"
4,Transformed data shape,"(712, 53)"
5,Transformed train set shape,"(498, 53)"
6,Transformed test set shape,"(214, 53)"
7,Numeric features,2
8,Categorical features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8213,0.8811,0.7270,0.7847,0.7515,0.6127,0.6170,1.3060
ridge,Ridge Classifier,0.8152,0.8790,0.6841,0.7990,0.7342,0.5942,0.6008,0.3020
knn,K Neighbors Classifier,0.8053,0.8424,0.6361,0.8047,0.7083,0.5657,0.5761,0.2100
et,Extra Trees Classifier,0.8032,0.8468,0.6570,0.7860,0.7130,0.5654,0.5728,0.3460
rf,Random Forest Classifier,0.7771,0.8417,0.6243,0.7439,0.6725,0.5067,0.5157,0.6900
svm,SVM - Linear Kernel,0.7610,0.8225,0.7152,0.6821,0.6874,0.4959,0.5069,0.2160
catboost,CatBoost Classifier,0.7591,0.7952,0.5649,0.7266,0.6118,0.4515,0.4675,1.6160
xgboost,Extreme Gradient Boosting,0.7489,0.7817,0.5670,0.7102,0.6135,0.4374,0.4521,0.2700
gbc,Gradient Boosting Classifier,0.7389,0.7078,0.5432,0.6752,0.5902,0.4088,0.4185,0.3060
ada,Ada Boost Classifier,0.7388,0.7582,0.5920,0.6726,0.6204,0.4253,0.4324,0.2860


Processing:   0%|          | 0/69 [00:00<?, ?it/s]